In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')
from frameworks.CPLELearning import CPLELearningModel

In [3]:
import pandas as pd
import sklearn
from xgboost import XGBClassifier

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
DF_data = pd.read_csv("../data/creditcard.csv")
DF_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
y = DF_data['Class']
X = DF_data.drop('Class', axis=1).values

In [6]:
index = range(0, len(DF_data), 100)
X_mini = DF_data.iloc[index].values
y_mini = y[index].values
y_mini = pd.np.random.choice([-1, 0, 1], len(X_mini))

In [ ]:
y_mini.mean()

-0.005967005967005967

In [ ]:
model = CPLELearningModel(XGBClassifier(), predict_from_probabilities=True)
model.fit(X_mini, y_mini.astype(int))

............nn

In [6]:
model = CPLELearningModel(sklearn.svm.SVC(kernel='linear', max_iter=10, probability=True), predict_from_probabilities=True)
model.fit(X_mini, y_mini)

/home/md/git-projects/semisup-learn/cple/local/lib/python2.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


ValueError: Found array with 0 sample(s) (shape=(0, 31)) while a minimum of 1 is required.

In [8]:
%%time

model = sklearn.svm.SVC(kernel='linear', probability=True, max_iter=10)
model.fit(DF_data_mini.values, y_mini.values)

CPU times: user 391 ms, sys: 460 µs, total: 391 ms
Wall time: 391 ms


/home/md/git-projects/semisup-learn/cple/local/lib/python2.7/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [9]:
dupa = model.predict_proba(DF_data_mini)

In [10]:
dupa

array([[0.99846466, 0.00153534],
       [0.99846466, 0.00153534],
       [0.99846466, 0.00153534],
       ...,
       [0.99846471, 0.00153529],
       [0.99846471, 0.00153529],
       [0.99846471, 0.00153529]])

In [13]:
model = RandomForestClassifier(100)
model.fit(DF_data_mini.values, y_mini.values)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)